In [ ]:
!pip install trl==0.13.0
!pip install datasets==3.2.0
!pip install -U bitsandbytes==0.45.0
!pip install git+https://github.com/huggingface/peft.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from datasets import load_dataset
from datasets import Dataset
from pprint import pprint
import transformers
from peft import prepare_model_for_kbit_training
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import json
import pickle
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
data_list = pickle.load(open('전자금융거래.pk', 'rb'))

In [ ]:
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
data = Dataset.from_list([{"text" : i} for i in data_list], split="train")

data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
data

Map:   0%|          | 0/75 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 75
})

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,  # LoRA 스케일링 계수로, 모델 학습 중 LoRA의 영향력을 조절
    lora_dropout=0.05,  # 드롭아웃 확률 설정
    bias="none",  # 바이어스 사용 설정 (여기서는 비활성화)
    use_dora=True,  # DoRA 학습을 활성화하는 옵션
    task_type="CAUSAL_LM"  # 생성 모델 학습을 지정
)

model = get_peft_model(model, config)

In [ ]:
input_text = "전자지급수단이 뭐야?"
gened = model.generate(
      **tokenizer(
          input_text,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=128001,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [ ]:
print(tokenizer.decode(gened[0]))

<|begin_of_text|>전자지급수단이 뭐야?」라고 묻는 질문에, 그는「어떤 조건하에선 전자지급수가 증가할 수 있다고 생각합니다. 예를 들어, 기계의 생산 비용이 줄어나는 조건하에선 전자지급수가 증가할 수 있습니다.」라고 답했다. 이는 전자지급수와 기계의 생산 비용 간의 복잡한 상호관계를 간접적으로 설명하고 있음을 의미한다. 그는 이 말을 통해 전자지급수와 기계의 생산 비용 간의 상호관계를 이해하고, 이를 통해 전자지급수의 증가 가능성을 높이는 방법을 탐색하고 있다는 것을 의미한다.이와 같이, 그는 전자지급수와 기계의 생산 비용 간의 상호관계를 이해하고, 이를 통해 전자지급수의 증가 가능성을 높이는 방법을 탐색하는 전략적인 접근 방식을 제시하고 있다.이로써, 전자지급수와 기계의 생산 비용 간의 상호관계를 이해하고, 이를 통해 전자지급수의 증가 가능성을 높이는 전략적인 접근 방식을 제시하고 있다는 것을 의미한다.이와 같이, 그는 전


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
data_len = data.num_rows

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=200,
        num_train_epochs = 3,
        # max_steps= int(data_len/batch_size) , # 1 epoch 만 학습
        learning_rate=1e-4,
        fp16=True,
        logging_steps=10,
        output_dir="qlora_output",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:482: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_aut

Step,Training Loss
10,2.155700
20,2.137000
30,2.084200
40,1.986100
50,1.940300
60,1.821800
70,1.771200
80,1.590500
90,1.460500
100,1.409900


In [ ]:
input_text = "전자지급수단이 뭐야?"
gened = model.generate(
      **tokenizer(
          input_text,
          return_tensors='pt',
          return_token_type_ids=False
      ).to(model.device),
      max_new_tokens=256,
      early_stopping=True,
      do_sample=True,
      eos_token_id=128001,
)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1797: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_

In [ ]:
print(tokenizer.decode(gened[0]))

<|begin_of_text|>전자지급수단이 뭐야?”, “신용카드의 위험성”, “전자지급수단의 법률적 문제” 등에 대해 다루고 있다. 
전자지급수단의 법률적 문제를 다루는 장에서는 “전자지급수단의 법률적 문제”라는 제목으로, “전자지급수단의 법률적 문제”라는 부제를 달성하고, “전자지급수단의 법률적 문제”라는 내용을 다룬다. 
전자지급수단의 법률적 문제를 다루는 장의 내용은 “전자지급수단의 법률적 문제”라는 제목으로, “전자지급수단의 법률적 문제”라는 부제를 달성하고, “전자지급수단의 법률적 문제”라는 내용을 다룬다. 
전자지급수단의 법률적 문제를 다루는 장의 내용은 “전자지급수단의 법률적 문제”라는 제목으로, “전자지급수단의 법률적 문제”라는 부제를 달성하고, “전자지급수단의 법률적 문제”라는 내용을 다룬다. 
전자
